In [1]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [2]:
dataset_path = "./datasets/sell-in.txt"
productos_a_predecir = "./datasets/product_id_apredecir201912.txt"

In [3]:
df = pd.read_csv(dataset_path, sep='\t')

In [4]:
df['product_id'] = df['product_id'].astype(str)
df['periodo'] = df['periodo'].astype(str)

In [5]:
df = df[~df['periodo'].isin(['201912'])]

In [6]:
df_a_predecir = pd.read_csv(productos_a_predecir, dtype={ 'product_id': str })

In [7]:
df['timestamp'] = pd.to_datetime(df['periodo'], format='%Y%m')

In [8]:
df = df[df['product_id'].isin(df_a_predecir['product_id'].unique())]

In [9]:
df_monthly_product = df.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [10]:
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [11]:
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [12]:
ts_data = ts_data.sort_index()

In [13]:
ts_data = ts_data.fill_missing_values()

In [14]:
predictor = TimeSeriesPredictor(
    prediction_length=1,
    target='tn',
    freq='MS'
)

In [15]:
predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Users\dylan\Desktop\labo3-2025v\AutogluonModels\ag-20250720_143708'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.21
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
GPU Count:          1
Memory Avail:       18.44 GB / 27.66 GB (66.7%)
Disk Space Avail:   228.56 GB / 699.94 GB (32.7%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provided train_dat

In [16]:
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [17]:
forecast

,,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
item_id,timestamp,,,,,,,,,,
20001,2019-12-01,1433.170230,1073.648815,1203.069860,1295.197429,1355.174614,1433.879765,1504.034564,1581.885127,1684.095883,1831.753717
20002,2019-12-01,1265.645587,898.309875,1025.597941,1127.269718,1185.096250,1266.296874,1341.402720,1409.828584,1517.897511,1674.818779
20003,2019-12-01,750.464335,523.114817,604.073071,658.547362,701.339054,751.102677,800.639128,836.146696,911.858352,1002.384901
20004,2019-12-01,580.800335,391.529781,458.985217,501.131848,542.890915,581.288996,623.568921,658.715353,720.410385,792.872095
20005,2019-12-01,540.602933,353.297537,419.520768,461.259685,504.341277,541.148710,584.110740,622.813716,686.033141,760.338838
...,...,...,...,...,...,...,...,...,...,...,...
21263,2019-12-01,-0.001411,-0.084277,-0.050224,-0.031138,-0.018675,-0.001359,0.015496,0.032575,0.059651,0.104375
21265,2019-12-01,0.038601,-0.059094,-0.021293,0.003433,0.019625,0.038694,0.057064,0.076999,0.107674,0.158814
21266,2019-12-01,0.036252,-0.054850,-0.018988,0.003508,0.019902,0.036346,0.053943,0.075731,0.102145,0.151419


In [18]:
forecast_mean = forecast['mean'].reset_index()
forecast_mean.columns

Index(['item_id', 'timestamp', 'mean'], dtype='object')

In [19]:
resultado = forecast['mean'].reset_index()
#resultado = resultado[resultado['timestamp'] == '2020-02-01']

In [20]:
resultado = resultado[['item_id', 'timestamp', 'mean']]
resultado.columns = ['product_id', 'periodo', 'tn']
resultado

,product_id,periodo,tn
0,20001,2019-12-01,1433.170230
1,20002,2019-12-01,1265.645587
2,20003,2019-12-01,750.464335
3,20004,2019-12-01,580.800335
4,20005,2019-12-01,540.602933
...,...,...,...
775,21263,2019-12-01,-0.001411
776,21265,2019-12-01,0.038601
777,21266,2019-12-01,0.036252
778,21267,2019-12-01,0.026097


In [ ]:
resultado.to_csv("forecast_autogluon_dic.csv", index=False)
resultado.head()

,product_id,periodo,tn
0,20001,2019-12-01,1433.170230
1,20002,2019-12-01,1265.645587
2,20003,2019-12-01,750.464335
3,20004,2019-12-01,580.800335
4,20005,2019-12-01,540.602933


: 